<a href="https://colab.research.google.com/github/johir-bd/Machine-Learning-Project/blob/master/Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time

class SelfDrivingCar:
    def __init__(self):
        self.speed = 0  # in km/h
        self.distance_to_obstacle = 100  # distance to nearest obstacle in meters

    def sense_environment(self):
        """ Simulate sensing the environment """
        # Randomly change distance to obstacle
        self.distance_to_obstacle = random.randint(20, 100)

    def decide_speed(self):
        """ Basic decision-making: adjust speed based on obstacle distance """
        if self.distance_to_obstacle < 30:
            self.speed = 0  # Stop if too close to an obstacle
            return "Obstacle detected! Stopping the car."
        elif 30 <= self.distance_to_obstacle < 60:
            self.speed = 30  # Slow down
            return "Obstacle ahead! Slowing down."
        else:
            self.speed = 60  # Drive at normal speed
            return "All clear! Driving at normal speed."

    def drive(self, iterations=10):
        """ Simulate driving decision-making for a number of iterations """
        output = []
        for _ in range(iterations):
            self.sense_environment()
            decision = self.decide_speed()
            output.append(f"Current speed: {self.speed} km/h, Distance to obstacle: {self.distance_to_obstacle} meters. {decision}")
            time.sleep(0.5)  # Slow down output for readability
        return output

# Run the simulation
car = SelfDrivingCar()
results = car.drive(iterations=10)

# Print results
for result in results:
    print(result)

Current speed: 60 km/h, Distance to obstacle: 78 meters. All clear! Driving at normal speed.
Current speed: 30 km/h, Distance to obstacle: 37 meters. Obstacle ahead! Slowing down.
Current speed: 60 km/h, Distance to obstacle: 96 meters. All clear! Driving at normal speed.
Current speed: 60 km/h, Distance to obstacle: 96 meters. All clear! Driving at normal speed.
Current speed: 60 km/h, Distance to obstacle: 76 meters. All clear! Driving at normal speed.
Current speed: 30 km/h, Distance to obstacle: 32 meters. Obstacle ahead! Slowing down.
Current speed: 60 km/h, Distance to obstacle: 80 meters. All clear! Driving at normal speed.
Current speed: 60 km/h, Distance to obstacle: 86 meters. All clear! Driving at normal speed.
Current speed: 30 km/h, Distance to obstacle: 58 meters. Obstacle ahead! Slowing down.
Current speed: 60 km/h, Distance to obstacle: 96 meters. All clear! Driving at normal speed.


In [2]:
!pip install gym


In [3]:


import gym

# Create a simple environment, here CartPole is used as an example
env = gym.make('CartPole-v1')
env.reset()

# Run a simple episode
for _ in range(1000):
    env.render()  # This usually displays the environment, not usable in Colab
    action = env.action_space.sample()  # Take a random action
    env.step(action)  # Execute the action

env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don

In [4]:
import gym

# Create the CartPole environment
env = gym.make('CartPole-v1')

# Observe the initial state
state = env.reset()
print("Initial state:", state)

# Close the environment
env.close()

Initial state: [-0.02160355 -0.0331586  -0.0092422   0.0469237 ]


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [5]:
import numpy as np

# Number of episodes to run
num_episodes = 5

for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Render the environment
        env.render()

        # Choose a random action
        action = env.action_space.sample()

        # Step the environment using the action
        state, reward, done, info = env.step(action)
        total_reward += reward

    print(f"Episode {episode + 1}: Total Reward: {total_reward}")

# Close the environment
env.close()

Episode 1: Total Reward: 44.0
Episode 2: Total Reward: 25.0
Episode 3: Total Reward: 15.0
Episode 4: Total Reward: 26.0
Episode 5: Total Reward: 26.0


In [8]:
import numpy as np
import gym

# Create the CartPole environment
env = gym.make('CartPole-v1')

# Parameters
num_episodes = 1000
learning_rate = 0.1
discount_factor = 0.95
exploration_prob = 1.0
exploration_decay = 0.995
min_exploration_prob = 0.01

# Discretizing the state parameters
def discretize_state(state):
    # Define bins for each observation (4 observations in CartPole)
    bins = [
        np.linspace(-2.4, 2.4, 10),  # Position of the cart (x)
        np.linspace(-3.0, 3.0, 10),  # Velocity of the cart (x')
        np.linspace(-0.5, 0.5, 10),  # Angle of the pole (theta)
        np.linspace(-2.0, 2.0, 10)   # Velocity of the pole (theta')
    ]
    # Digitize the state into bin indices, corrected to fit Q-table dimensions
    discretized = []
    for i in range(len(state)):
        index = np.digitize(state[i], bins[i])
        # Ensure the index fits in the Q-table dimension
        index = min(index, len(bins[i]) - 1)  # cap the index to prevent overflow
        discretized.append(index)
    return tuple(discretized)

# Size of Q-table based on the number of bins across all dimensions
q_table = np.zeros((10, 10, 10, 10, env.action_space.n))

# Train the agent
for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Choose action (exploration or exploitation)
        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()  # Explore
        else:
            # Exploit: select the action with max Q value
            action = np.argmax(q_table[discretize_state(state)])

        # Step the environment
        new_state, reward, done, info = env.step(action)
        total_reward += reward

        # Update Q-value
        best_future_q = np.max(q_table[discretize_state(new_state)])
        q_table[discretize_state(state)][action] += learning_rate * (reward + discount_factor * best_future_q - q_table[discretize_state(state)][action])

        state = new_state

    # Decay exploration probability
    exploration_prob = max(min_exploration_prob, exploration_prob * exploration_decay)

    if episode % 100 == 0:
        print(f"Episode {episode}: Total Reward: {total_reward}, Exploration Probability: {exploration_prob:.2f}")

# Close the environment
env.close()

Episode 0: Total Reward: 12.0, Exploration Probability: 0.99
Episode 100: Total Reward: 53.0, Exploration Probability: 0.60
Episode 200: Total Reward: 20.0, Exploration Probability: 0.37
Episode 300: Total Reward: 46.0, Exploration Probability: 0.22
Episode 400: Total Reward: 24.0, Exploration Probability: 0.13
Episode 500: Total Reward: 29.0, Exploration Probability: 0.08
Episode 600: Total Reward: 45.0, Exploration Probability: 0.05
Episode 700: Total Reward: 44.0, Exploration Probability: 0.03
Episode 800: Total Reward: 34.0, Exploration Probability: 0.02
Episode 900: Total Reward: 54.0, Exploration Probability: 0.01


In [9]:
# Evaluate the agent
for episode in range(5):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        env.render()
        action = np.argmax(q_table[discretize_state(state)])  # Choose the best action
        state, reward, done, info = env.step(action)
        total_reward += reward

    print(f"Evaluation Episode {episode + 1}: Total Reward: {total_reward}")

# Close the environment
env.close()

Evaluation Episode 1: Total Reward: 62.0
Evaluation Episode 2: Total Reward: 30.0
Evaluation Episode 3: Total Reward: 29.0
Evaluation Episode 4: Total Reward: 41.0
Evaluation Episode 5: Total Reward: 45.0
